In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
import os, shutil, random, glob

import pandas as pd
import sys
import gdal
import imp

from shutil import copyfile

import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params


Using TensorFlow backend.
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

EG
13
MA
13
ZA
13
CN
13
IN
13
JP
13
KP
13
PK
13
PH
13
TH
13
CR
13
TR
13
AT
13
BE
13
BG
13
HR
13
DK
13
FI
13
FR
13
DE
13
GR
13
IE
13
IT
13
LT
13
NL
13
NO
13
PL
13
PT
13
RO
13
ES
13
SE
13
GB
13
UA
13
CA
13
MX
13
US
13
AU
13
NZ
13
BR
13
CL
13
UY
13
MIX
13
GLOBAL
13
EG
18
MA
18
ZA
18
CN
18
IN
18
JP
18
KP
18
PK
18
PH
18
TH
18
CR
18
TR
18
AT
18
BE
18
BG
18
HR
18
DK
18
FI
18
FR
18
DE
18
GR
18
IE
18
IT
18
LT
18
NL
18
NO
18
PL
18
PT
18
RO
18
ES
18
SE
18
GB
18
UA
18
CA
18
MX
18
US
18
AU
18
NZ
18
BR
18
CL
18
UY
18
MIX
18
GLOBAL
18
EG
19
MA
19
ZA
19
CN
19
IN
19
JP
19
KP
19
PK
19
PH
19
TH
19
CR
19
TR
19
AT
19
BE
19
BG
19
HR
19
DK
19
FI
19
FR
19
DE
19
GR
19
IE
19
IT
19
LT
19
NL
19
NO
19
PL
19
PT
19
RO
19
ES
19
SE
19
GB
19
UA
19
CA
19
MX
19
US
19
AU
19
NZ
19
BR
19
CL
19
UY
19
MIX
19
GLOBAL
19
EG
13
MA
13
ZA
13
CN
13
IN
13
JP
13
KP
13
PK
13
PH
13
TH
13
CR
13
TR
13
AT
13
BE
13
BG
13
HR
13
DK
13
FI
13
FR
13
DE
13
GR
13
IE
13
IT
13
LT
13
NL
13
NO
13
PL
13
PT
13
RO
13
ES
13
SE
13
GB
13
UA
13
CA
13
MX
13
U

In [7]:
# COUNTRIES = ['AT', 'BR', 'CN', 'FR', 'DE']
COUNTRIES = ['AT', 'BR', 'CN', 'FR', 'DE']
RESOLUTION = 13

#BR, FR, CN

tgt_dir_tb = get_param("MIX", "PATH_RAW_IMAGES_TURBINES", RESOLUTION)
tgt_dir_notb = get_param("MIX","PATH_RAW_IMAGES_NOTURBINES", RESOLUTION)

shutil.rmtree(tgt_dir_tb,ignore_errors=True)
shutil.rmtree(tgt_dir_notb,ignore_errors=True)

os.makedirs(tgt_dir_tb)
os.makedirs(tgt_dir_notb)


quality_check_list=[]

# COUNTRIES = ['AT']

In [28]:
for COUNTRY in COUNTRIES:
    
    print("=================== "+COUNTRY+" ======================")
    src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES_MACHINE_CLASSIFIED")
    src_dir_notb = get_param(COUNTRY,"PATH_RAW_IMAGES_NOTURBINES")
    src_dir_notb_png = get_param(COUNTRY,"PATH_RAW_IMAGES_NOTURBINES_MACHINE_CLASSIFIED")
    
    src_dir_tb_png_osm = fpr.get_param(COUNTRY, "PATH_RAW_IMAGES_OSM") + "hard_to_assess/"
    src_dir_notb_png_osm = fpr.get_param(COUNTRY, "PATH_RAW_IMAGES_OSM") + "no_turbines/"

    files_tb_png_osm = []
    files_notb_png_osm = []

    
    files_tb = [x for x in os.listdir(src_dir_tb) if x.endswith(".png")]
    files_notb = [x for x in os.listdir(src_dir_notb) if x.endswith(".tif")]
    files_notb_png = [x for x in os.listdir(src_dir_notb_png) if x.endswith(".png")]
    
    if(os.path.exists(src_dir_tb_png_osm)):
        files_tb_png_osm = [x for x in os.listdir(src_dir_tb_png_osm) if x.endswith(".png")]
    
    if(os.path.exists(src_dir_notb_png_osm)):
        files_notb_png_osm = [x for x in os.listdir(src_dir_notb_png_osm) if x.endswith(".png")]
    
    print("files turbine: " + str(len(files_tb)))
    print("files no turbine tif: " + str(len(files_notb)))
    print("files no turbine png: " + str(len(files_notb_png)))
    
    print("files turbine png osm: " + str(len(files_tb_png_osm)))
    print("files no turbine png osm: " + str(len(files_notb_png_osm)))
    
    
    
    for i in range(len(files_tb)):
        
        #if i % 20 == 0:
        #    print("Current file: "+str(i))
        
        try:
        
            f_tb = files_tb[i]
            file_name_tb = f_tb[0:-4] + "_" + COUNTRY + f_tb[-4:len(f_tb)]
            copyfile(src_dir_tb + f_tb, tgt_dir_tb + file_name_tb)
            
        except:
            print("copy error")
            print(src_dir_tb+f_tb)
            print(tgt_dir_tb+file_name_tb)
        
        quality_check_list.append([file_name_tb, 100])    
        
    for i in range(len(files_notb)):
        
        #if i % 20 == 0:
        #    print("Current file: "+str(i))
        
        try:
            #print(tgt_dir_notb + f_notb[0:-4] + "_" + COUNTRY + ".png")
            f_notb = files_notb[i]
            src = src_dir_notb + f_notb
            tgt = tgt_dir_notb + f_notb[0:-4] + "_" + COUNTRY + ".png"
            fpr.copyfile_to_png(src, tgt)
            
        
        except Exception as e:
            print(e)
            print("copy error")
            print(src_dir_notb+files_notb[i])
            print(tgt_dir_notb+files_notb[i][0:-4]+"_"+COUNTRY+".png")
            
            
    for i in range(len(files_tb_png_osm)):
        
        #if i % 20 == 0:
        #    print("Current file: "+str(i))
        
        try:
        
            f_tb_png = files_tb_png_osm[i]
            file_name_tb_png = f_tb_png[0:-4] + "_" + COUNTRY + f_tb_png[-4:len(f_tb_png)]
            copyfile(src_dir_tb_png_osm + f_tb_png, tgt_dir_tb + file_name_tb_png)
            
        except Exception as e:
            print(e)
            print("copy error")
            print(src_dir_tb_png_osm + files_tb_png[i])
            print(tgt_dir_tb + f_tb_png[0:-4] + "_" + COUNTRY + f_tb_png[-4:len(f_tb)])
        
        quality_check_list.append([file_name_tb_png, 100])    
        
            
      
    
    for i in range(len(files_notb_png)):
        
        #if i % 20 == 0:
        #    print("Current file: "+str(i))
        
        try:
        
            f_notb_png = files_notb_png[i]
            file_name_notb_png = f_notb_png[0:-4] + "_" + COUNTRY + f_notb_png[-4:len(f_notb_png)]
            copyfile(src_dir_notb_png + f_notb_png, tgt_dir_notb + file_name_notb_png)
            
        except Exception as e:
            print(e)
            print("copy error")
            print(src_dir_notb_png + files_notb_png[i])
            print(tgt_dir_notb + f_notb_png[0:-4] + "_" + COUNTRY + f_notb_png[-4:len(f_tb)])
            
    for i in range(len(files_notb_png_osm)):
        
        #if i % 20 == 0:
        #    print("Current file: "+str(i))
        
        try:
        
            f_notb_png = files_notb_png_osm[i]
            file_name_notb_png = f_notb_png[0:-4] + "_" + COUNTRY + f_notb_png[-4:len(f_notb_png)]
            copyfile(src_dir_notb_png_osm + f_notb_png, tgt_dir_notb + file_name_notb_png)
            
        except Exception as e:
            print(e)
            print("copy error")
            print(src_dir_notb_png_osm + files_notb_png[i])
            print(tgt_dir_notb + f_notb_png[0:-4] + "_" + COUNTRY + f_notb_png[-4:len(f_tb)])
            
            

    dir = tgt_dir_notb

    for xmlpath in glob.iglob(os.path.join(dir, '*.xml')):
        os.remove(xmlpath)
    
        
        
            
    
    
quality_check = pd.DataFrame(quality_check_list,columns=['id','quality'])
 

    

=================== AT ======================
files turbine: 871
files no turbine tif: 1122
files no turbine png: 0
files turbine png osm: 0
files no turbine png osm: 0
=================== BR ======================
files turbine: 1230
files no turbine tif: 9859
files no turbine png: 0
files turbine png osm: 0
files no turbine png osm: 0
=================== CN ======================
files turbine: 3534
files no turbine tif: 0
files no turbine png: 3268
files turbine png osm: 0
files no turbine png osm: 0
=================== FR ======================
files turbine: 78
files no turbine tif: 0
files no turbine png: 390
files turbine png osm: 0
files no turbine png osm: 0
=================== DE ======================
files turbine: 0
files no turbine tif: 0
files no turbine png: 0
files turbine png osm: 784
files no turbine png osm: 1863


In [9]:
 fpr.get_param(COUNTRY, "PATH_RAW_IMAGES_OSM")

'data/aerialImages/GOOGLE/RESOLUTION19/DE/osm/'

In [11]:
src_dir_tb_png_osm

'data/aerialImages/GOOGLE/RESOLUTION19/DE/osm/hard_to_assess'

In [29]:
outp_file = get_param("MIX","FILE_QUALITY_CHECK")

f = open(outp_file, 'w')

f.write(quality_check.to_csv().encode('utf-8', 'replace').decode('utf-8'))

quality_check.shape



(6497, 2)